<a href="https://colab.research.google.com/github/ByteBelle-21/maze_solver/blob/main/maze_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from google.colab import files
import cv2
from gym import Env
from gym.spaces import Discrete, Box
import random


In [ ]:
is_valid = False
filename = ""
while not is_valid :
  input_image = files.upload()
  filename = next(iter(input_image))
  is_valid = cv2.haveImageWriter(filename)
  if not is_valid :
    print("File format not supported. Upload new file!")
  else :
    opened_file = open(filename, "wb")
    opened_file.write(input_image[filename])

valid_image = cv2.imread(filename)
if valid_image is None:
  print("File doesn't exist")
  exit(0)
plt.imshow(valid_image)
plt.axis("off")
plt.title("Original Uploaded Image")
plt.show()

In [ ]:
gray_image = cv2.cvtColor(valid_image, cv2.COLOR_BGR2GRAY)
binary_image = cv2.adaptiveThreshold(gray_image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 15, 2)
plt.imshow(binary_image, cmap='gray')
plt.axis("off")
plt.title("Updated Binary Image")
plt.show()

In [ ]:
image_matrix = (binary_image == 255).astype(int)
print("Converted image matrix")
print(image_matrix)

In [ ]:
def opening_top_border(image_matrix):
  height, width = image_matrix.shape
  opening = []
  for x in range(height):
    wall_found = False
    for y in range(width):
      if image_matrix[x][y] == 1 and wall_found:
        opening.append((x, y))
        if y == (width - 1):
          return []
      elif image_matrix[x][y] == 0 and not wall_found:
        wall_found = True
      elif image_matrix[x][y] == 0 and wall_found and image_matrix[x][y-1] == 1:
        return opening
  return opening

def opening_bottom_border(image_matrix):
  height, width = image_matrix.shape
  opening = []
  for x in reversed(range(height)):
    wall_found = False
    for y in range(width):
      if image_matrix[x][y] == 1 and wall_found:
        opening.append((x, y))
        if y == (width - 1):
          return []
      elif image_matrix[x][y] == 0 and not wall_found:
        wall_found = True
      elif image_matrix[x][y] == 0 and wall_found and image_matrix[x][y-1] == 1:
        return opening
  return opening


def opening_left_border(image_matrix):
  height, width = image_matrix.shape
  opening = []
  for y in range(width):
    wall_found = False
    for x in range(height):
      if image_matrix[x][y] == 1 and wall_found:
        opening.append((x, y))
        if x == (height - 1):
          return []
      elif image_matrix[x][y] == 0 and not wall_found:
        wall_found = True
      elif image_matrix[x][y] == 0 and wall_found and image_matrix[x-1][y] == 1:
        return opening
  return opening

def opening_right_border(image_matrix):
  height, width = image_matrix.shape
  opening = []
  for y in reversed(range(width)):
    wall_found = False
    for x in range(height):
      if image_matrix[x][y] == 1 and wall_found:
        opening.append((x, y))
        if x == (height - 1):
          return []
      elif image_matrix[x][y] == 0 and not wall_found:
        wall_found = True
      elif image_matrix[x][y] == 0 and wall_found and image_matrix[x-1][y] == 1:
        return opening
  return opening

In [ ]:
top_opening = opening_top_border(image_matrix)
bottom_opening = opening_bottom_border(image_matrix)
left_opening = opening_left_border(image_matrix)
right_opening  =  opening_right_border(image_matrix)

start = (-1, -1)
end = (-1, -1)

if top_opening == [] and bottom_opening == [] and left_opening == [] and right_opening == []:
  print("No opening found")
  exit(0)

if len(top_opening) > 0:
  if start == (-1, -1):
    start = (top_opening[0][0],((top_opening[0][1] + top_opening[-1][1]) // 2))
  elif end == (-1, -1):
    end = (top_opening[0][0],((top_opening[0][1] + top_opening[-1][1]) // 2))

if len(bottom_opening) > 0:
  if start == (-1, -1):
    start = (bottom_opening[0][0],((bottom_opening[0][1] + bottom_opening[-1][1]) // 2))
  elif end == (-1, -1):
    end = (bottom_opening[0][0],((bottom_opening[0][1] + bottom_opening[-1][1]) // 2))

if len(left_opening) > 0:
  if start == (-1, -1):
    start = (((left_opening[0][0] + left_opening[-1][0]) // 2), left_opening[0][1])
  elif end == (-1, -1):
    end = (((left_opening[0][0] + left_opening[-1][0]) // 2), left_opening[0][1])

if len(right_opening) > 0:
  if start == (-1, -1):
    start = (((right_opening[0][0] + right_opening[-1][0]) // 2), right_opening[0][1])
  elif end == (-1, -1):
    end = (((right_opening[0][0] + right_opening[-1][0]) // 2), right_opening[0][1])


print("Start position is", start)
print("End position is", end)

In [ ]:
class mazeEnv(Env):
  def __init__(self, image_matrix, start, end):
    self.image_matrix = image_matrix
    self.start = start
    self.end = end
    self.goal = end
    self.position = start
    self.action_space = Discrete(4)
    self.observation_space = Box(low=0, high=1, shape=(image_matrix.shape[0], image_matrix.shape[1], 1), dtype=np.uint8)

  def step(self, action):
    x, y =  self.position
    if action == 0:
      x -= 1
    elif action == 1:
      x += 1
    elif action == 2:
      y += 1
    elif action == 3:
      y -= 1

    if 0 <= x < self.image_matrix.shape[0] and 0 <= y < self.image_matrix.shape[1] and self.image_matrix[x][y] == 1:
      self.position = (x, y)
      reward = 0
    else:
      reward = -1

    if self.position == self.goal:
      reward = 10
    return self.image_matrix, reward, self.position == self.goal, {}

  def reset(self):
    self.position = self.start
    self.goal = self.end
    return self.image_matrix
